## Programmer salaries

In [1]:
import pandas as pd
import numpy as np
import string

In [2]:
df = pd.read_csv('../data/so_2021_survey_results.csv',
                 usecols=['LanguageHaveWorkedWith',
                          'LanguageWantToWorkWith',
                          'Country',
                          'CompTotal'])


few setups.

In [25]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83439 entries, 0 to 83438
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country                 83439 non-null  object 
 1   CompTotal               47183 non-null  float64
 2   LanguageHaveWorkedWith  82357 non-null  object 
 3   LanguageWantToWorkWith  76821 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.5+ MB


 What are the different programming languages that developers currently use?

In [5]:
df['LanguageHaveWorkedWith'].str.lower().dropna().str.split(';').explode().value_counts().index

Index(['javascript', 'html/css', 'python', 'sql', 'java', 'node.js',
       'typescript', 'c#', 'bash/shell', 'c++', 'php', 'c', 'powershell', 'go',
       'kotlin', 'rust', 'ruby', 'dart', 'assembly', 'swift', 'r', 'vba',
       'matlab', 'groovy', 'objective-c', 'scala', 'perl', 'haskell', 'delphi',
       'clojure', 'elixir', 'lisp', 'julia', 'f#', 'erlang', 'apl', 'crystal',
       'cobol'],
      dtype='object', name='LanguageHaveWorkedWith')

What are the 10 programming languages most commonly used today?

In [6]:
worked_with = df['LanguageHaveWorkedWith'].str.lower().dropna().str.split(';').explode().value_counts().head(10).index
worked_with

Index(['javascript', 'html/css', 'python', 'sql', 'java', 'node.js',
       'typescript', 'c#', 'bash/shell', 'c++'],
      dtype='object', name='LanguageHaveWorkedWith')

What are the 10 programming languages people most want to use?

In [7]:
want_to_work_with = df['LanguageWantToWorkWith'].str.lower().dropna().str.split(';').explode().value_counts().head(10).index
want_to_work_with

Index(['javascript', 'python', 'html/css', 'typescript', 'sql', 'node.js',
       'c#', 'java', 'rust', 'go'],
      dtype='object', name='LanguageWantToWorkWith')

What languages are on both top 10 lists?

In [8]:
# thoes values need to be indexed. or not we cant compare. 

want_to_work_with.intersection(worked_with)


Index(['javascript', 'python', 'html/css', 'typescript', 'sql', 'node.js',
       'c#', 'java'],
      dtype='object')

What languages in the top 10 have people worked with but dont want to work with in the future?

In [9]:
worked_with[~want_to_work_with.isin(worked_with)]


Index(['bash/shell', 'c++'], dtype='object', name='LanguageHaveWorkedWith')

What is the most popular (current) language used by people in each country?

In [10]:
all_languages = df['LanguageHaveWorkedWith'].str.split(';').explode()
all_languages.head()

0            C++
0       HTML/CSS
0     JavaScript
0    Objective-C
0            PHP
Name: LanguageHaveWorkedWith, dtype: object

In [11]:
df[['Country']].join(all_languages).groupby('Country').agg(pd.Series.mode).head()

,LanguageHaveWorkedWith
Country,
Afghanistan,JavaScript
Albania,JavaScript
Algeria,JavaScript
Andorra,JavaScript
Angola,"[HTML/CSS, JavaScript]"


 What is the mean number of languages used in the last year?


In [12]:
df['LanguageHaveWorkedWith'].str.split(';').str.len().mean()

np.float64(5.373678011583714)

What is the greatest number of languages people listed as having used in the last year?

In [13]:
df['LanguageHaveWorkedWith'].str.split(';').str.len().max()

np.float64(38.0)

How many people chose that largest number?

In [14]:
df.loc[df['LanguageHaveWorkedWith'].str.split(';').str.len() == 38,'LanguageHaveWorkedWith'].count()

np.int64(32)

How many people in the survey claim salaries of $2 million or more?

In [15]:
df.loc[df['CompTotal'] >= 2_000_000]['CompTotal'].count()

np.int64(2369)

In [16]:
#  remove them from our data, because otherwise it will be skewed

df = df.loc[df['CompTotal'] < 2_000_000]

**NOTE:** We’ll get back to salaries in a moment. Now we take the LanguageHaveWorkedWith column and turn it into multiple columns to so we can analyze the individual languages more easily. Doing this is known as creating dummy columns. Instead of a column containing the string 'JavaScript;Python', we create one column called JavaScript and another called Python, putting 1s where the person marked themselves as using JavaScript and 0s where they indicated they did not.

In [21]:
df['LanguageHaveWorkedWith'].str.get_dummies(sep=';').head(10)

,APL,Assembly,Bash/Shell,C,C#,C++,COBOL,Clojure,Crystal,Dart,Delphi,Elixir,Erlang,F#,Go,Groovy,HTML/CSS,Haskell,Java,JavaScript,Julia,Kotlin,LISP,Matlab,Node.js,Objective-C,PHP,Perl,PowerShell,Python,R,Ruby,Rust,SQL,Scala,Swift,TypeScript,VBA
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0
9,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
11,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
12,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
16,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0
18,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
24,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
26,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [23]:
# now time to add it in the main data frame

df = pd.concat([df, df['LanguageHaveWorkedWith'].str.get_dummies(sep=';')], axis='columns')
df.head(10)

,Country,CompTotal,LanguageHaveWorkedWith,LanguageWantToWorkWith,APL,Assembly,Bash/Shell,C,C#,C++,COBOL,Clojure,Crystal,Dart,Delphi,Elixir,Erlang,F#,Go,Groovy,HTML/CSS,Haskell,Java,JavaScript,Julia,Kotlin,LISP,Matlab,Node.js,Objective-C,PHP,Perl,PowerShell,Python,R,Ruby,Rust,SQL,Scala,Swift,TypeScript,VBA
0,Slovakia,"4,800.00",C++;HTML/CSS;JavaScript;Objective-C;PHP;Swift,Swift,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0
9,Sweden,"42,000.00",C++;Python,Haskell;Python,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
11,Spain,"43,000.00",Bash/Shell;HTML/CSS;JavaScript;Node.js;SQL;Typ...,C++;Clojure;JavaScript;Node.js;Rust;SQL;TypeSc...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
12,Germany,"71,500.00",C;C++;Java;Perl;Ruby,Rust,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
16,Turkey,"9,000.00",C#;HTML/CSS;Java;JavaScript;Node.js,C#;Java;JavaScript;Node.js,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
17,Canada,"61,000.00",Bash/Shell;HTML/CSS;JavaScript;PHP;Ruby;SQL,Bash/Shell;HTML/CSS;JavaScript;PHP;Ruby;SQL,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0
18,Singapore,"18,700.00",C++;Python,C++;Python,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
23,France,"36,000.00",JavaScript;Python,Python;SQL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
24,Germany,"72,000.00",C++;HTML/CSS;Java;JavaScript;Kotlin;Node.js;Ty...,HTML/CSS;Java;JavaScript;Kotlin;Node.js;TypeSc...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
26,Switzerland,"80,000.00",C++;Python,C++;Go;Java;Python,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


The average salary of someone who knows Python and JavaScript but not Java

In [27]:
df['CompTotal'][(df['Python'] == 1) & (df['JavaScript'] == 1) & (df['Java'] == 0)].mean()

np.float64(126817.99470235605)

Someone who knows Python and Java but not JavaScript?

In [28]:
df['CompTotal'][(df['Python'] == 1) & (df['JavaScript'] == 0) & (df['Java'] == 1)].mean()

np.float64(162737.10379596677)

Someone who knows Java and JavaScript but not Python.

In [29]:
df['CompTotal'][(df['Python'] == 0) & (df['JavaScript'] == 1) & (df['Java'] == 1)].mean()

np.float64(140867.65981559738)

When developers are stuck (as indicated in the column NEWStuck), what are the three things they’re most likely to do?

In [34]:
df_ = pd.read_csv('../data/so_2021_survey_results.csv', low_memory= False)

In [35]:
df_['NEWStuck'].str.lower().dropna().str.split(';').explode().value_counts(ascending= False).head(3)

NEWStuck
google it                            74491
visit stack overflow                 66410
do other work and come back later    39871
Name: count, dtype: int64

What proportion of the survey respondents marked their gender as Man? Does that proportion seem similar to your real-life experiences?

In [37]:
df_[['Gender']].value_counts(normalize=True)

Gender                                                                            
Man                                                                                  0.91
Woman                                                                                0.05
Prefer not to say                                                                    0.02
Non-binary, genderqueer, or gender non-conforming                                    0.01
Or, in your own words:                                                               0.01
Man;Or, in your own words:                                                           0.00
Man;Non-binary, genderqueer, or gender non-conforming                                0.00
Woman;Non-binary, genderqueer, or gender non-conforming                              0.00
Man;Woman                                                                            0.00
Non-binary, genderqueer, or gender non-conforming;Or, in your own words:             0.00
Man;Woman;Non-bin

On average, what proportion of their years coding have been done professionally?

In [38]:
df_.loc[df_['YearsCode'] == 'Less than 1 year', 'YearsCode'] = 0
df_.loc[df_['YearsCode'] == 'More than 50 years', 'YearsCode'] = 51

df_.loc[df_['YearsCodePro'] == 'Less than 1 year', 'YearsCodePro'] = 0
df_.loc[df_['YearsCodePro'] == 'More than 50 years', 'YearsCodePro'] = 51

# Turn them into integers
df_ = df_[['YearsCode', 'YearsCodePro']].dropna()
df_['YearsCode'] = df_['YearsCode'].astype(np.int16)
df_['YearsCodePro'] = df_['YearsCodePro'].astype(np.int16)

# Get rid of rows with 0 YearsCode
df_ = df_[df_['YearsCode'] != 0]

(df_['YearsCodePro'] / df_['YearsCode']).mean()

np.float64(0.5923711657118932)